Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [ ]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import st_loss
import time
import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
path = "../"
category = "real_data"
files = glob2.glob(f'{path}{category}/*.h5')
files = [f[len(f"'{path}{category}"):-3] for f in files]
files

In [ ]:
sczi = pd.read_pickle(f"../output/pickle_results/real_data/real_data_sczi.pkl")

In [ ]:
df = pd.DataFrame()
dropout = 0.9
lr = 1e-5
layers = [200, 40, 60]
temperature = 0.07
for dataset in files:

    print(f">>>>> Data {dataset}")
    print("SCZI ", sczi[sczi["dataset"] == dataset]["ARI"].mean())
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")

    
    nb_genes = 500
    data_mat = h5py.File(f"{path}real_data/{dataset}.h5", "r")
    X = np.array(data_mat['X'])
    Y = np.array(data_mat['Y'])
    cluster_number = np.unique(Y).shape[0]

    X = train.preprocess(X, nb_genes=nb_genes)
    for train_size in [ 1, 0.8, 0.6, 0.4, 0.2
    ]:
        for run in range(3):
            torch.manual_seed(run)
            torch.cuda.manual_seed_all(run)
            np.random.seed(run)
            random.seed(run)
            if train_size == 1:
                X_train = X
                y_train = Y
            else:
                X_train, X_test, y_train, y_test = train_test_split(
                    X, Y, train_size=train_size, random_state=run)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            start = time.time()
            dresults = train.run(X_train,
                                 cluster_number,
                                 dataset,
                                 Y=y_train,
                                 nb_epochs=30,
                                 lr=lr,
                                 temperature=temperature,
                                 dropout=dropout,
                                 evaluate=False,
                                 n_ensemble=1,
                                 layers=layers,
                                 save_to=f"{path}output/{category}/{run}/",
                                 save_pred=True)
            elapsed = time.time() - start
            dresults["temperature"] = temperature
            dresults["dropout"] = dropout
            dresults["train_size"] = train_size
            dresults["layers"] = str(layers)
            dresults["run"] = run
            dresults["time"] = elapsed
            #         print(f".", end = "")
            #         print(f"# {temperature}, {dropout}, {lr}, {layers}",
            #               dresults.get('COMBINED_kmeans_ari', ""),
            #               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
            #               dresults.get('leiden_ari_0', ""))
            df = df.append(dresults, ignore_index=True)

            df.to_pickle(
                f"{path}output/pickle_results/{category}/{category}_train_size.pkl"
            )

In [ ]:
X_train.shape

In [ ]:
df.groupby("train_size").mean()

In [ ]:
df.mean()#0.686020 20 # 0.631504:10 # 0.693487 : 30 #0.685507